In [1]:
import numpy as np
import pandas as pd
import emoji
from glove_vec import read_glove_vectors

from keras.models import Sequential
from keras.layers import Dense, Input, Dropout, SimpleRNN, LSTM, Activation
from keras.utils import np_utils

import matplotlib.pyplot as plt

Using TensorFlow backend.
/home/inderpreet/.local/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/home/inderpreet/.local/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/home/inderpreet/.local/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/home/inderpreet/.local/lib/python3.6/site-packages/t

In [2]:
train = pd.read_csv("train_emoji.csv", header=None)
test = pd.read_csv("test_emoji.csv", header=None)

In [3]:
train.head()

,0,1,2,3
0,never talk to me again,3,NaN,NaN
1,I am proud of your achievements,2,NaN,NaN
2,It is the worst day in my life,3,NaN,NaN
3,Miss you so much,0,NaN,[0]
4,food is life,4,NaN,NaN


In [4]:
test.head()

,0,1
0,I want to eat\t,4
1,he did not answer\t,3
2,he got a raise\t,2
3,she got me a present\t,0
4,ha ha ha it was so funny\t,2


In [5]:
emoji_dict = {0 : ":heart:", 1 : ":baseball:", 2 : ":smile:", 
              3 : ":disappointed:", 4 : ":fork_and_knife:"}

In [6]:
for ix in emoji_dict.keys():
    print (ix, emoji.emojize(emoji_dict[ix],use_aliases=True))

0 ❤
1 ⚾
2 😄
3 😞
4 🍴


In [7]:
X_train = train[0]
Y_train = train[1]

X_test = test[0]
Y_test = test[1]

print (X_train.shape, Y_train.shape)
print (X_test.shape, Y_test.shape)

(132,) (132,)
(56,) (56,)


In [9]:
print (X_train[0], Y_train[0])

never talk to me again 3


In [8]:
for ix in range(X_train.shape[0]):
    X_train[ix] = X_train[ix].split()

for ix in range(X_test.shape[0]):
    X_test[ix] = X_test[ix].split()

Y_train = np_utils.to_categorical(Y_train)

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """


In [9]:
print (X_train[0], Y_train[0])

['never', 'talk', 'to', 'me', 'again'] [0. 0. 0. 1. 0.]


In [10]:
np.unique(np.array([len(ix) for ix in X_train]), return_counts=True)

(array([ 1,  2,  3,  4,  5,  6,  7,  8,  9, 10]),
 array([ 4,  5, 26, 35, 20, 21, 11,  5,  1,  4]))

In [11]:
np.unique(np.array([len(ix) for ix in X_test]), return_counts=True)

(array([2, 3, 4, 5, 6, 7, 8]), array([ 3, 12, 16, 17,  3,  4,  1]))

In [14]:
emb_matrix = read_glove_vectors("/home/vasu/all_projects/CB/glove.6B.50d.txt")

In [15]:
print (emb_matrix["i"], emb_matrix["i"].shape)

[ 1.1891e-01  1.5255e-01 -8.2073e-02 -7.4144e-01  7.5917e-01 -4.8328e-01
 -3.1009e-01  5.1476e-01 -9.8708e-01  6.1757e-04 -1.5043e-01  8.3770e-01
 -1.0797e+00 -5.1460e-01  1.3188e+00  6.2007e-01  1.3779e-01  4.7108e-01
 -7.2874e-02 -7.2675e-01 -7.4116e-01  7.5263e-01  8.8180e-01  2.9561e-01
  1.3548e+00 -2.5701e+00 -1.3523e+00  4.5880e-01  1.0068e+00 -1.1856e+00
  3.4737e+00  7.7898e-01 -7.2929e-01  2.5102e-01 -2.6156e-01 -3.4684e-01
  5.5841e-01  7.5098e-01  4.9830e-01 -2.6823e-01 -2.7443e-03 -1.8298e-02
 -2.8096e-01  5.5318e-01  3.7706e-02  1.8555e-01 -1.5025e-01 -5.7512e-01
 -2.6671e-01  9.2121e-01] (50,)


In [16]:
embeddings_train = np.zeros((X_train.shape[0], 10, 50))
embeddings_test = np.zeros((X_test.shape[0], 10, 50))

for ix in range(X_train.shape[0]):
    for ij in range(len(X_train[ix])):
        embeddings_train[ix][ij] = emb_matrix[X_train[ix][ij].lower()]
        

for ix in range(X_test.shape[0]):
    for ij in range(len(X_test[ix])):
        embeddings_test[ix][ij] = emb_matrix[X_test[ix][ij].lower()]

In [19]:
print (embeddings_train.shape, embeddings_test.shape)

(132, 10, 50) (56, 10, 50)


In [47]:
model = Sequential()
model.add(LSTM(32, input_shape=(10,50), return_sequences=True))
model.add(Dropout(0.5))
model.add(LSTM(32, return_sequences=False))
model.add(Dropout(0.5))
model.add(Dense(5, activation='softmax'))

model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm_5 (LSTM)                (None, 10, 32)            10624     
_________________________________________________________________
dropout_11 (Dropout)         (None, 10, 32)            0         
_________________________________________________________________
lstm_6 (LSTM)                (None, 32)                8320      
_________________________________________________________________
dropout_12 (Dropout)         (None, 32)                0         
_________________________________________________________________
dense_6 (Dense)              (None, 5)                 165       
Total params: 19,109
Trainable params: 19,109
Non-trainable params: 0
_________________________________________________________________


In [48]:
model.compile(loss="categorical_crossentropy", optimizer="RMSprop", metrics=['accuracy'])

In [49]:
hist = model.fit(embeddings_train, Y_train, epochs=50, batch_size=12,shuffle=True, validation_split=0.05)

Train on 125 samples, validate on 7 samples
Epoch 1/50
125/125 [==============================] - 2s 18ms/step - loss: 1.6043 - acc: 0.2080 - val_loss: 1.6428 - val_acc: 0.1429
Epoch 2/50
125/125 [==============================] - 0s 1ms/step - loss: 1.5572 - acc: 0.3280 - val_loss: 1.6317 - val_acc: 0.1429
Epoch 3/50
125/125 [==============================] - 0s 1ms/step - loss: 1.4968 - acc: 0.4080 - val_loss: 1.6410 - val_acc: 0.1429
Epoch 4/50
125/125 [==============================] - 0s 1ms/step - loss: 1.4568 - acc: 0.3600 - val_loss: 1.6293 - val_acc: 0.1429
Epoch 5/50
125/125 [==============================] - 0s 1ms/step - loss: 1.3851 - acc: 0.4080 - val_loss: 1.5798 - val_acc: 0.1429
Epoch 6/50
125/125 [==============================] - 0s 2ms/step - loss: 1.3240 - acc: 0.4720 - val_loss: 1.4916 - val_acc: 0.2857
Epoch 7/50
125/125 [==============================] - 0s 3ms/step - loss: 1.2772 - acc: 0.4800 - val_loss: 1.4080 - val_acc: 0.4286
Epoch 8/50
125/125 [===========

In [51]:
pred = model.predict_classes(embeddings_test)

In [52]:
acc = float(sum(pred==Y_test))/embeddings_test.shape[0]

In [53]:
print(acc)

0.6071428571428571


In [54]:
def print_emoji(label):
    
    return emoji.emojize(emoji_dict[label], use_aliases=True)

In [55]:
for ix in range(embeddings_test.shape[0]):
    
    if pred[ix] != Y_test[ix]:
        print (X_test[ix], print_emoji(Y_test[ix]), print_emoji(pred[ix]))

['he', 'got', 'a', 'raise'] 😄 ❤
['I', 'am', 'upset'] ❤ 😞
['We', 'had', 'such', 'a', 'lovely', 'dinner', 'tonight'] ❤ 😄
['This', 'girl', 'is', 'messing', 'with', 'me'] 😞 ❤
['are', 'you', 'serious', 'ha', 'ha'] 😄 😞
['work', 'is', 'horrible'] 😞 😄
['Congratulation', 'for', 'having', 'a', 'baby'] 😄 ❤
['stop', 'messing', 'around'] 😞 ⚾
['you', 'brighten', 'my', 'day'] 😄 ❤
['she', 'is', 'a', 'bully'] 😞 ❤
['I', 'worked', 'during', 'my', 'birthday'] 😞 ❤
['enjoy', 'your', 'break'] 😄 ⚾
['valentine', 'day', 'is', 'near'] ❤ 😄
['My', 'life', 'is', 'so', 'boring'] 😞 ❤
['I', 'like', 'to', 'laugh'] 😄 ❤
['I', 'will', 'go', 'dance'] 😄 ⚾
['I', 'like', 'your', 'jacket'] 😄 ❤
['Good', 'job'] 😄 ❤
['I', 'want', 'to', 'joke'] 😄 ❤
['go', 'away'] 😞 ⚾
['yesterday', 'we', 'lost', 'again'] 😞 ⚾
['I', 'did', 'not', 'have', 'breakfast'] 😞 ❤
